The game [Wordle](https://www.powerlanguage.co.uk/wordle/) continues to be a fun game with friends. The [NY Times wrote that Wordle is a Love Story](https://www.nytimes.com/2022/01/03/technology/wordle-word-game-creator.html). We're not solving Wordle because we don't believe in love stories. We're solving Wordle because math is also a love story, and the underlying patterns found in our every day life are beautiful. We're solving Wordle because we think AI is something that should be explainable and interpretable. We think that all love stories start with trust.

*Wordle rules:*

1. Guess the WORDLE in 6 tries.

1. Each guess must be a valid 5 letter word. Hit the enter button to submit.

1. After each guess, the color of the tiles will change to show how close your guess was to the word.

The strategy is to make a word guess by reducing the number of alternative possibilities. We reduce alternative possibilities by starting with words that include the most vowels, then choosing words that include matching values with the most popular consonants. We repeat the cycle of choose vowels, choose popular consonants until hopefully finding a solution.


### Credits

Note that the strategy is from Tiffany Chen. She introduced me to Wordle and coached me on how she's been playing the game. I did my best to listen and translate her intuition into an algorithm to solve Wordle games.


### References

A great example of constraint propagation is [Peter Norvig solving all the Sudoku puzzles](https://norvig.com/sudoku.html). The [OR-Tools Constraint Optimization guide from Google](https://developers.google.com/optimization/cp) is a learning opportunity for me. We can make a more general connection to the [boolean satisfiability problem](https://en.wikipedia.org/wiki/Boolean_satisfiability_problem). As a higher level goal, I would like to understand if these approaches can help improve AI interpretability/explainability.

### Dependencies and Constants

Keep track of these for maintainability of code.

In [1]:
from collections import defaultdict
import json
import random
import math

In [2]:
CONSONANTS = set(['b','c','d','f','g','h','j','k','l','m','n','p','q','r','s','t','v','w','x','z'])
VOWELS = set(['a','e','i','o','u','y'])

## Wordle Words

Wordle words are from the word list found by [Robert Reichel, (Reverse Engineering Wordle)](https://reichel.dev/blog/reverse-engineering-wordle.html). I retrieved the list by using Chrome Dev Tools, parsing the Javascript for the `var La` array. I also found a `Ta` array containing a second list of words. Reichel appears to focus on the `La` array so I do the same here. Screenhots of parsing the Javascript for the word lists:

![la_wordle_words](var_la.png)

![ta_wordle_words](var_ta.png)

In [3]:
with open("la.json", "r") as f:
    data = json.load(f)

## Wordle Game

Replicating the wordle game so we can run simulations.

In [4]:
def wordle_word(data, seed=42):
    """ Word of the day. """
    random.seed(seed)
    return data[random.randint(0,len(data)-1)]
    

In [5]:
def play_wordle(guess:str, word_of_the_day:str) -> list:
    board = []
    correct_letters = set(word_of_the_day)
    has_letters = set(guess)
    has_letters = has_letters & correct_letters
    for correct_letter, guessed_letter in zip(word_of_the_day, guess):
        if correct_letter == guessed_letter:
            board.append(guessed_letter)
        else:
            board.append("")
    return board, has_letters
        

In [6]:
example_wordle_word = wordle_word(data)
example_guess = "adieu"
print("example guess '{}''\nexample wordle '{}'".format(example_guess, example_wordle_word))

example guess 'adieu''
example wordle 'onset'


In [7]:
play_wordle(example_guess, example_wordle_word)

(['', '', '', 'e', ''], {'e'})

## Global Distribution of Vowels & Consonants

We want to maximize the number of correct letters guessed even if 
the order is not apparent. We do this by generating frequencies of
vowels and consonants from the Wordle words. Our general strategy
is to narrow down candidates by leveraging the global distribution
of letters within Wordle words.

In [8]:
def most_shared(data:list, has_vowels=True):
    possible = []
    letters = defaultdict(int)
    shared = CONSONANTS
    if has_vowels:
        shared = VOWELS
    for word in data:
        wordset = set(word)
        overlap = len(wordset & shared)
        if overlap > 0:
            possible.append((word, overlap))
        for letter in (wordset & shared):
            letters[letter] += 1
            
    letters = {k:v for k,v in sorted(letters.items(), key=lambda x: x[1], reverse=True)}
    possible = sorted(possible, key=lambda x: x[1], reverse=True)
    return possible, letters

In [9]:
possible_vowels, usage_vowels = most_shared(data, has_vowels=True)
possible_vowels[:5]

[('bayou', 4), ('audio', 4), ('quiet', 3), ('adobe', 3), ('repay', 3)]

In [10]:
possible_conson, usage_conson = most_shared(data, has_vowels=False)
possible_conson[:5]

[('blush', 4), ('dwarf', 4), ('stink', 4), ('bench', 4), ('fresh', 4)]

In [11]:
usage_vowels

{'e': 1056, 'a': 909, 'o': 673, 'i': 647, 'u': 457, 'y': 417}

## Establish a Set of Candidates

We use all of the Wordle words as our starting set of candidates.

In [12]:
# Establish set of candidates
cands = {k: set(k) for k in data}

len(cands)

2315

## Assignment Strategy

We're assigning guesses based on most frequent vowels, followed by most frequent consonants. In the event that we guess the correct position of a letter, we will also assign based on that letter position within the game board.

In [13]:
def break_ties(cands, usage):
    # break ties by scoring with most frequent vowels
    uv = {x[0]: idx for idx, x in enumerate(reversed(usage_vowels.items())) }
    for bucket, words in cands.items():
        scored = []
        for word in words:
            scored.append((word, sum([uv.get(i,0) for i in word])))
        scored = [i[0] for i in sorted(scored, key=lambda x: x[1], reverse=True)]
        cands[bucket] = scored
    return cands
    
def feature_bucket(cands:dict, usage:dict, has_vowels=True) -> list:
    buckets = defaultdict(list)
    cset = CONSONANTS
    if has_vowels:
        cset = VOWELS
    for word, wordset in cands.items():
        buckets[len(wordset & cset)].append(word)
    buckets = break_ties(buckets, usage)
    return buckets 

In [14]:
feature_bucket(cands, usage_vowels, has_vowels=True)[4][:5]

['audio', 'bayou']

In [15]:
 play_wordle("aeiou", example_wordle_word)

(['', '', '', '', ''], {'e', 'o'})

In [16]:
board, found = play_wordle("audio", example_wordle_word)
board, found

(['', '', '', '', ''], {'o'})

In [17]:
# We can also get information using the contra-positive
set("audio") - found

{'a', 'd', 'i', 'u'}

Here we can see that just inputting vowels instead of a real word gets us
further along then choosing the word with the most vowels. We broke the tie
between words using most vowels by sorting within buckets by most frequent 
vowels.

## Constraint Propagation

1. If a word does not contain found letters, eliminate it from candidates.
1. Eliminate letters based on shared position.
1. If ties are found, break them using most frequent letters.

In [18]:
def eliminate(cands:dict, game:list, found:set, contra:set) -> dict:
    candidate_words = list(cands.keys())
    
    for candidate_word in candidate_words:
        
        wordset = cands[candidate_word]
        
        # (1a) Eliminate based on shared letters
        if len(wordset & found) != len(found):
            cands.pop(candidate_word)
            continue
            
        # (1b) Eliminate contra-positive shared letters
        if len(wordset & contra) > 0:
            cands.pop(candidate_word)
            continue
            
        # (2) Eliminate based on shared position
        for i in range(len(game)):
            if len(game[i]) > 0 and candidate_word[i] != game[i]:
                cands.pop(candidate_word)
                break
                
    return cands

In [19]:
# test elimination step

test_cands = {
 'cigar': {'a', 'c', 'g', 'i', 'r'},
 'rebut': {'b', 'e', 'r', 't', 'u'},
 'siste': {'i', 's', 't','e'},
 'humph': {'h', 'm', 'p', 'u'},
}

test_guess = "cigar"
test_wordle = "siste"
test_game = ['','i','','','']
test_found = set(['i'])
test_contra = set(test_guess) - test_found

eliminate(test_cands, test_game, test_found, set(test_guess) - test_found)

{'siste': {'e', 'i', 's', 't'}}

## Playing Wordle

We've established relevant distributions. Let's play Wordle.

In [20]:
def choose_guess(buckets:dict) -> str:
    choose = max(list(buckets.keys()))
    return buckets[choose][0]

def solved_game(board:list):
    return all([len(i) > 0 for i in board])

def report_attempt(board:list, found:set, guess_attempt:int, cands:dict):
    print("Guess attempt: {}".format(guess_attempt))
    print("\tboard: {}".format(board))
    print("\tfound: {}".format(found))
    print("\tRemaining candidates: {}".format(len(cands)))

def report_solution(board:list, guess_attempt:int, report=True):
    h = {
        "board" : "".join(board),
        "attempts" : guess_attempt,
        "solved": True,
    }
    if report:
        print("Solved game.")
        print(h)
    return h

def compute_wordle(cands:dict, example_wordle_word:str, usage_vowels:dict, usage_cons:dict, 
                   max_tries=6, report=True):
    
    if report:
        print("Starting candidates %d" % len(cands))
    
    wordle_guess_attempt = 1
    while wordle_guess_attempt <= max_tries:
        
        # (1) Find vowels
        vowel_bucket = feature_bucket(cands, usage_vowels, has_vowels=True)
        guess_word = choose_guess(vowel_bucket)
        board, found = play_wordle(guess_word, example_wordle_word)
        contra = set(guess_word) - found
        if solved_game(board):
            return report_solution(board, wordle_guess_attempt, report)
        
        cands = eliminate(cands, board, found, contra)
        if report:
            report_attempt(board, found, wordle_guess_attempt, cands)
            print("\tguess: %s" % guess_word)
        wordle_guess_attempt += 1
        
        # (2) Find consonants
        conson_bucket = feature_bucket(cands, usage_vowels, has_vowels=False)
        guess_word = choose_guess(conson_bucket)
        board, found = play_wordle(guess_word, example_wordle_word)
        contra = set(guess_word) - found
        if solved_game(board):
            return report_solution(board, wordle_guess_attempt, report)
        
        cands = eliminate(cands, board, found, contra)
        if report:
            report_attempt(board, found, wordle_guess_attempt, cands)
            print("\tguess: %s" % guess_word)
        wordle_guess_attempt += 1
        
    h = {
        "board" : "".join(board),
        "attempts" : wordle_guess_attempt,
        "solved": False,
    }
        
    if report:
        print("Unable to solve game")
        print(h)
        
    return h

In [21]:
compute_wordle(cands, example_wordle_word, usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: {'o'}
	Remaining candidates: 295
	guess: audio
Guess attempt: 2
	board: ['', '', '', '', '']
	found: {'o', 't'}
	Remaining candidates: 25
	guess: forth
Guess attempt: 3
	board: ['', '', '', 'e', '']
	found: {'e', 'o', 't'}
	Remaining candidates: 5
	guess: totem
Solved game.
{'board': 'onset', 'attempts': 4, 'solved': True}


{'board': 'onset', 'attempts': 4, 'solved': True}

## Running Simulations

Simulate Wordle games.

In [22]:
len(data)

2315

In [23]:
def wordle_simulations(data:list, n:int, seed:int, usage_vowels:dict,
                      usage_cons:dict, max_tries:int, report:bool):
    random.seed(seed)
    wordle_words = random.sample(data, n)
    
    status = []
    for wordle_word in wordle_words:
        
        try:
            
            # Establish set of candidates
            cands = {k: set(k) for k in data}
    
            result = compute_wordle(cands, wordle_word, usage_vowels, 
                                usage_cons, max_tries, report)
        
            status.append((wordle_word, result))
            
        except Exception as exc:
            status.append((wordle_word, str(exc)))
        
    return status
    

In [24]:
n_simulations = 1000
simulation_results = wordle_simulations(data, n_simulations, 42, usage_vowels, usage_conson, 6, report=False)

In [25]:
count = 0
for w,g in simulation_results:
    if type(g) is dict and g['solved']:
        count += 1
        
count / n_simulations

0.926

In [26]:
chances = defaultdict(int)
for w, res in simulation_results:
    if type(res) is dict and res["solved"]:
        chances[res["attempts"]] += 1
chances

defaultdict(int, {4: 322, 3: 194, 6: 117, 5: 252, 2: 40, 1: 1})

## Benchmarking 

Doing a manual check against recent Wordles.

In [27]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "perky", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: set()
	Remaining candidates: 190
	guess: audio
Guess attempt: 2
	board: ['', 'e', '', '', '']
	found: {'e'}
	Remaining candidates: 39
	guess: bench
Guess attempt: 3
	board: ['', 'e', '', '', 'y']
	found: {'e', 'y', 'r'}
	Remaining candidates: 5
	guess: leery
Solved game.
{'board': 'perky', 'attempts': 4, 'solved': True}


{'board': 'perky', 'attempts': 4, 'solved': True}

In [28]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "mount", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: {'u', 'o'}
	Remaining candidates: 70
	guess: audio
Guess attempt: 2
	board: ['m', 'o', 'u', '', 't']
	found: {'m', 'u', 'o', 't'}
	Remaining candidates: 1
	guess: moult
Solved game.
{'board': 'mount', 'attempts': 3, 'solved': True}


{'board': 'mount', 'attempts': 3, 'solved': True}

In [29]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "sugar", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', 'u', '', '', '']
	found: {'a', 'u'}
	Remaining candidates: 20
	guess: audio
Solved game.
{'board': 'sugar', 'attempts': 2, 'solved': True}


{'board': 'sugar', 'attempts': 2, 'solved': True}

In [30]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "tiger", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: {'i'}
	Remaining candidates: 296
	guess: audio
Guess attempt: 2
	board: ['', '', '', '', '']
	found: {'i', 't'}
	Remaining candidates: 51
	guess: stink
Guess attempt: 3
	board: ['', 'i', '', '', '']
	found: {'i', 'e', 't'}
	Remaining candidates: 8
	guess: piety
Solved game.
{'board': 'tiger', 'attempts': 4, 'solved': True}


{'board': 'tiger', 'attempts': 4, 'solved': True}

In [31]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "could", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: {'u', 'o', 'd'}
	Remaining candidates: 17
	guess: audio
Guess attempt: 2
	board: ['', 'o', 'u', '', 'd']
	found: {'u', 'o', 'd'}
	Remaining candidates: 3
	guess: pound
Solved game.
{'board': 'could', 'attempts': 3, 'solved': True}


{'board': 'could', 'attempts': 3, 'solved': True}

In [32]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "wrung", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: {'u'}
	Remaining candidates: 200
	guess: audio
Guess attempt: 2
	board: ['', '', 'u', '', '']
	found: {'u'}
	Remaining candidates: 11
	guess: blush
Guess attempt: 3
	board: ['', 'r', 'u', '', '']
	found: {'u', 'r'}
	Remaining candidates: 2
	guess: erupt
Solved game.
{'board': 'wrung', 'attempts': 4, 'solved': True}


{'board': 'wrung', 'attempts': 4, 'solved': True}

In [33]:
cands = {k: set(k) for k in data}
compute_wordle(cands, "light", usage_vowels, usage_conson)

Starting candidates 2315
Guess attempt: 1
	board: ['', '', '', '', '']
	found: {'i'}
	Remaining candidates: 296
	guess: audio
Guess attempt: 2
	board: ['', '', '', '', '']
	found: {'i', 't'}
	Remaining candidates: 51
	guess: stink
Guess attempt: 3
	board: ['', 'i', '', '', '']
	found: {'i', 't'}
	Remaining candidates: 14
	guess: piety
Solved game.
{'board': 'light', 'attempts': 4, 'solved': True}


{'board': 'light', 'attempts': 4, 'solved': True}